### Reatime response Speech Recognition 
#### for Google Speech Commands 35 words

In [1]:
import sounddevice as sd
import numpy as np

nChannel= 1
nSamplePerSec= sampleRate= 16000
nSamplePerFrame= 1000
nFramePerSec= nSamplePerSec//nSamplePerFrame # 16 frame/sec

indexFrame= 0
bufferTime= 10 #sec
nFramePerBuffer= nFramePerSec* bufferTime #160 # frames == 10 sec
BufferSize= nFramePerBuffer


ryBuffer= (1e-10)*np.random.random((BufferSize, nSamplePerFrame, nChannel))

def ryClearBuffer():
    global ryBuffer, indexFrame
    indexFrame=0
    ryBuffer= (1e-10)*np.random.random((BufferSize, nSamplePerFrame, nChannel))

def ryCallback(indata, outdata, frames, time, status):
    global indexFrame, ryBuffer

    if status:
        print(status)
        
    ## for sound playback
    outdata[:] = indata  # *2

    ryBuffer[indexFrame%BufferSize]= indata       
    indexFrame += 1


import time

import pylab as pl

import ryLab01_1 as ry


def ryOpenStream(aCallback):
    
    aStream= sd.Stream(callback=    aCallback, 
               channels=   nChannel,       # 1 for mono, 2 for stereo
               samplerate= nSamplePerSec,  # sample/sec
               blocksize=  nSamplePerFrame #1000   # frame_size_in_sample, sample/frame
               )
    return aStream
    

def ryGet1secSpeech(withPrintInfo= False):
    global ryBuffer, BufferSize, indexFrame
    
    x= ryBuffer
    t1= (indexFrame%BufferSize)
    x= np.vstack((x[t1:], x[0:t1]))
    x= x.flatten()    
    x= x.astype(np.float32) 
    x= x[-16000:]    
    spEng= x.std()
    
    if withPrintInfo== True:
        print('[{:.1f}]({:.4f}), '.format(t, spEng), end='', flush=True)
    else:
        print('.', end='', flush=True)
    
    return x

def ryGet1secSpeechAndRecogItWithProb():
    
    x= ryGet1secSpeech()
    
    y, prob= ry.recWav(x, featureOut= False, withProb= True)
    
    return y, prob
        

In [4]:
spDuration= 1000 * bufferTime  # bufferTime= 10 seconds
recProbToConfirm= 0.8
ryClearBuffer()

from tqdm import tqdm

with ryOpenStream(ryCallback) as ryStream:
    
    t0= time.time()
    
    t=   0
    dt= .1 # sec  time_interval to do recognition
    loopNumber= 1000
    loopTime= loopNumber * dt
    
    nStop= 0
    nReallyStop= 10
    
    #while t<spDuration:
    #for i in tqdm(range(loopNumber)):
    for i in range(loopNumber):  
        
        t00= time.time()
        
        y, prob= ryGet1secSpeechAndRecogItWithProb()
        
        if prob > recProbToConfirm: #0.8:
            
            info= '【{}】@({:.1f})'.format(y, t)
            print(info, end='', flush=True)
        
            if y== 'stop':
                nStop+= 1
                
                info= '【【{}】】({}, Really？)'.format(y, nStop)
                print(info, end='\n', flush=True)
                
                if nStop >= nReallyStop:
                    
                    info= '【{}】(OK, I will STOP！！！)'.format(y)
                    print(info, end='\n', flush=True)
                    break
            
            if y== 'go':
                nStop= 0
                
                info= '【【{}】】(OK, Reset nStop= {}, and then GO)！！！'.format(y, nStop)
                print(info, end='\n', flush=True)
        
        dt00= time.time()-t00
        if dt-dt00>0:
            time.sleep(dt-dt00)
        
        t+=dt
        
    dtt= time.time() - t0
    print('dtt(sec)= {:.3f}'.format(dtt))

print('ry: 明けましておめでとう Happy New Year, 2020 !!!')

.【_silence_】@(0.0)..【tree】@(0.2)...............................【on】@(3.3).【on】@(3.4).....................【one】@(5.5).【one】@(5.6).【one】@(5.7).【one】@(5.8).【one】@(5.9).【one】@(6.0).【one】@(6.1).【one】@(6.2).【one】@(6.3).【one】@(6.4)....................................【two】@(10.0).【two】@(10.1).【two】@(10.2).【two】@(10.3).【two】@(10.4).【two】@(10.5).【two】@(10.6).【two】@(10.7).【two】@(10.8)..【eight】@(11.0).........................................【three】@(15.1)...【three】@(15.4).【three】@(15.5).【eight】@(15.6).【eight】@(15.7)input overflow, output underflow
.【three】@(15.8).【eight】@(15.9).【eight】@(16.0).【eight】@(16.1)......................................【two】@(19.9)..【two】@(20.1).【two】@(20.2).【four】@(20.3)..【go】@(20.5)【【go】】(OK, Reset nStop= 0, and then GO)！！！
.【go】@(20.6)【【go】】(OK, Reset nStop= 0, and then GO)！！！
.【four】@(20.7).【eight】@(20.8).【eight】@(20.9)............................【yes】@(23.7).【zero】@(23.8)..【learn】@(24.0)...【zero】@(24.3)...【no】@(24.6)..................【on】@(26.4).............【stop】@(27